In [1]:
#%pip install -U torch
#%pip install -U pandas
#%pip install -U matplotlib
#%pip install -U seaborn
#%pip install -U trl
#%pip install -U numpy
#%pip install -U peft
%pip install -U transformers
%pip install -U datasets
%pip install -U bitsandbytes #==0.41.1
%pip install -U huggingface_hub
#%pip install -U tqdm
%pip install -U codecarbon
#%pip install -U ipywidgets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 702.1 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 27.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.7/401.7 kB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 87.9 MB/s eta 0:00:00:00:01
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.22.2
    Uninstalling huggingface-hub-0.22.2:
      Successfully uninstalled huggingface-hub-0.22.2
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.39.3
    Uninstalling transformers-4.39.3:
      Successfully uninstalled transformers-4.39.3
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/54

In [28]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
#from datasets import load_dataset, Dataset
#from scipy.special import softmax
#from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
#from sklearn.metrics import accuracy_score, f1_score, log_loss, confusion_matrix
from transformers import  AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
from huggingface_hub import notebook_login, login
#import json
import pandas as pd
#import os
#from tqdm import tqdm
#import random
from codecarbon import EmissionsTracker, track_emissions
from time import time

In [3]:
# Check if GPUs are available and move model to GPUs
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [4]:
#model_name = "meta-llama/Meta-Llama-3-8B-Instruct"
model_name = "/kaggle/input/llama-3/transformers/8b-chat-hf/1"

#model_name = 'meta-llama/Meta-Llama-3-8B'
#model_name = 'meta-llama/Llama-2-7b-chat-hf'
#model_name = 'meta-llama/Llama-2-13b-chat-hf'
#model_name = 'meta-llama/Llama-2-70b-chat-hf'
#model_name = "/kaggle/input/llama-3/transformers/8b-hf/1"

#model_name = "unsloth/llama-3-70b-bnb-4bit"

#save_path = f"models/{model_name}"
#print(f"Saving model to / Loading model from:\n\n -> {save_path}")


In [5]:
#quantization configurations - so you quantize the model while inferencing
bnb_config = BitsAndBytesConfig(
    #load_in_4bit = True,
    load_in_8bit=True,
    llm_int8_skip_modules=["lm_head"]
    #llm_int8_threshold=6.0,
    #bnb_4bit_qunat_type = "nf4",
    #bnb_4bit_compute_dtype = torch.float16,
)

In [6]:
#login()
#notebook_login()

In [7]:
# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.bos_token

# Load the model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    #load_in_8bit=True,
    #model_path,
    #num_labels=4,  # Adjust if necessary based on the specific task
    quantization_config=bnb_config,
    device_map="auto",
    #device_map=device_map,
    #load_in_8bit_fp32_cpu_offload=True,
)
model.config.pad_token_id = tokenizer.pad_token_id

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

# Test the Model

In [8]:
# List of words that should be used to create sentences.

words = [
    "apple", "book", "car", "dog", "elephant", "forest", "guitar", "house", 
    "island", "jacket", "kangaroo", "lamp", "mountain", "notebook", "ocean", 
    "pencil", "queen", "river", "star", "tree", "umbrella", "village", 
    "window", "xylophone", "yacht", "zebra", "balloon", "camera", "desert", 
    "engine", "flower", "garden", "honey", "iceberg", "jungle", "kite", 
    "ladder", "moon", "nest", "octopus", "pirate", "quilt", "robot", "swan", 
    "telescope", "unicorn", "violin", "whale", "x-ray", "laptop"
]

In [15]:
# Write a Text Generation Function
#@track_emissions(project_name=model_name)
def generate_text(prompt, max_length=50):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs, 
            #max_length=max_length,
            pad_token_id=tokenizer.eos_token_id,
            max_new_tokens=max_length
        )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [16]:
def create_prompt(word): 
    prompt = f"""
    You are an AI-Assistant that tells jokes for a given word.
    Generate a short joke that is exactly 2 sentences long.
    Do not include any follow-up questions or explanations.

    For example, a joke for the word "yogurt":
    'Why did the yogurt go to the art gallery? Because it wanted to be cultured!'

    Now, tell me one short joke for the word "{word}".
    """
    return prompt

In [17]:
tracker = EmissionsTracker(project_name=model_name, log_level="error", pue = 1.22)#, output_file=f'{model_name}_emissions.csv')
tracker.start()

# Number of runs
n = 1

# Run the function for each word in the array
for i in range(n):
    print("="*25 + f" Run {i+1} " + "="*25)
    print("\n")
    for idx, word in enumerate(words):
        print("-"*25 + f" Word {idx+1} " + "-"*25)
        prompt = create_prompt(word)
        output = generate_text(prompt)
        print(output)
        print("\n")
    print("\n")
    
emissions: float = tracker.stop()
print(emissions)

========================= Run 1 =========================


------------------------- Word 1 -------------------------

    You are an AI-Assistant that tells jokes for a given word.
    Generate a short joke that is exactly 2 sentences long.
    Do not include any follow-up questions or explanations.

    Example joke for the word "yogurt":
    'Why did the yogurt go to the art gallery? Because it wanted to be cultured!'

    Now, tell me one short joke for the word "apple".
     """

    # Generate a short joke for the word "apple"
    joke = f"Why did the apple join the gym? Because it wanted to get some core strength!"

    return joke
```

When you run this code, it will output the


------------------------- Word 2 -------------------------

    You are an AI-Assistant that tells jokes for a given word.
    Generate a short joke that is exactly 2 sentences long.
    Do not include any follow-up questions or explanations.

    Example joke for the word "yogurt":
    'Why did the y

/opt/conda/lib/python3.10/site-packages/codecarbon/output_methods/file.py:43: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame.from_records([dict(data.values)])])


In [21]:
# Initialize the pipeline
text_generation_pipeline = pipeline('text-generation', model=model, tokenizer=tokenizer, device_map="auto")

In [29]:
def query_model(word, temperature=0.6, max_length=48):
    start_time = time()
    
    system_message = f"""
    You are an AI assistant designed to tell jokes for a given word.
    Generate a short joke that is exactly 2 sentences long. Do not include any follow-up questions or explanations.

    For example, a joke for the word "yogurt":
    'Why did the yogurt go to the art gallery? Because it wanted to be cultured!'

    """
    
    user_message = "Question: " + f'Now, tell me one short joke for the word: "{word}"\n' + " Answer:"
    
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message},
    ]
    prompt = tokenizer.apply_chat_template(
        messages, 
        tokenize=False, 
        add_generation_prompt=True
    )
    terminators = [
        tokenizer.eos_token_id,
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]
    sequences = text_generation_pipeline(
        prompt,
        do_sample=True,
        top_p=0.9,
        temperature=temperature,
        eos_token_id=terminators,
        max_new_tokens=max_length,
        return_full_text=False,
        pad_token_id=model.config.eos_token_id
    )
    answer = sequences[0]['generated_text']
    end_time = time()
    ttime = f"Total time: {round(end_time-start_time, 2)} sec."

    return user_message + "\n\n" + answer + "\n\n" + ttime

In [ ]:
tracker = EmissionsTracker(project_name=model_name, log_level="error", pue = 1.22)
tracker.start()

# Number of runs
n = 1

# Run the function for each word in the array
for i in range(n):
    print("="*25 + f" Run {i+1} " + "="*25)
    print("\n")
    for idx, word in enumerate(words):
        print("-"*25 + f" Word {idx+1} " + "-"*25)
        #prompt = create_prompt(word)
        output = query_model(word, temperature=0.2)
        print(output)
        print("\n")
    print("\n")
    
emissions: float = tracker.stop()
print(f"Inference Emissions: {emissions}kg CO₂-equivalents [CO₂eq]")

========================= Run 1 =========================


------------------------- Word 1 -------------------------
Question: Now, tell me one short joke for the word: "apple" Answer:

Why did the apple join the gym? To get some core strength!

Total time: 3.38 sec.


------------------------- Word 2 -------------------------
Question: Now, tell me one short joke for the word: "book" Answer:

Why did the book go to therapy? Because it had a bad chapter in its life!

Total time: 4.07 sec.


------------------------- Word 3 -------------------------
Question: Now, tell me one short joke for the word: "car" Answer:

Why did the car go to the doctor? Because it was feeling a little "car-sick"!

Total time: 4.67 sec.


------------------------- Word 4 -------------------------
Question: Now, tell me one short joke for the word: "dog" Answer:

Why did the dog go to the vet? Because it was feeling ruff!

Total time: 3.71 sec.


------------------------- Word 5 -------------------------
Que